<a href="https://colab.research.google.com/github/neerajkumarkannoujiya/11_JavaSpark/blob/main/Neeraj_copy_of_Final_version_BDA_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz
!pip install -q findspark

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [73.0 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [4,161 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
spark

In [ ]:
!wget https://archive.apache.org/dist/hadoop/common/hadoop-3.3.4/hadoop-3.3.4.tar.gz

In [ ]:
!tar -xzvf hadoop-3.3.4.tar.gz && cp -r hadoop-3.3.4/ /usr/local/

In [21]:
#To find the default Java path and add export in hadoop-env.sh
JAVA_HOME = !readlink -f /usr/lib/jvm/java-11-openjdk-amd64 | sed "s:bin/java::"
java_home_text_command = f"$ {JAVA_HOME[0]} "
!echo export JAVA_HOME=$java_home_text >>/home/neeraj7388011/hadoop/etc/hadoop/hadoop-env.sh

/bin/bash: line 1: /home/neeraj7388011/hadoop/etc/hadoop/hadoop-env.sh: No such file or directory


In [ ]:
import os
os.environ['HADOOP_HOME']="/home/neeraj7388011/hadoop"
os.environ['JAVA_HOME']=java_home_text

In [ ]:
!/home/neeraj7388011/hadoop/bin/hadoop
!/home/neeraj7388011/hadoop/bin/hadoop fs -mkdir elec2_repo
!/home/neeraj7388011/hadoop/bin/hadoop fs -copyFromLocal /content/elec2_* elec2_repo/
!/home/neeraj7388011/hadoop/bin/hadoop fs -ls elec2_repo

/bin/bash: line 1: /home/neeraj7388011/hadoop/bin/hadoop: No such file or directory
/bin/bash: line 1: /home/neeraj7388011/hadoop/bin/hadoop: No such file or directory
/bin/bash: line 1: /home/neeraj7388011/hadoop/bin/hadoop: No such file or directory
/bin/bash: line 1: /home/neeraj7388011/hadoop/bin/hadoop: No such file or directory


In [ ]:
! export HDFS_NAMENODE_USER="root"
! export HDFS_DATANODE_USER="root"
! export HDFS_SECONDARYNAMENODE_USER="root"
! export YARN_RESOURCEMANAGER_USER="root"
! export YARN_NODEMANAGER_USER="root"

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ElectricityDataProcessing").getOrCreate()
df = spark.read.option("delimiter", " ") .option("inferSchema", "true").csv("elec2_repo/elec2_data.dat", header=False)

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/content/elec2_repo/elec2_data.dat.

In [ ]:
df.show()

In [ ]:
column_names = ["date", "day", "period", "nswprice", "nswdemand", "vicprice", "vicdemand", "transfer"]
df = df.toDF(*column_names)
df.show()

In [ ]:
df_processed = df.drop("nswprice", "date")
df_processed.show()

In [ ]:
df_labels = spark.read.option("delimiter", " ") .option("inferSchema", "true").csv("elec2_repo/elec2_label.dat", header=False)
df_labels.show()

In [ ]:
!pip install tensorflow-io
!pip install kafka-python
import os
from datetime import datetime
import time
import threading
import json
from kafka import KafkaProducer
from kafka.errors import KafkaError
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_io as tfio
# Download Kafka (double-check the URL to ensure it is correct)
!curl -O https://downloads.apache.org/kafka/3.7.1/kafka_2.12-3.7.1.tgz

# Extract the tarball (once the file is confirmed to be correct)
!tar -xzf kafka_2.12-3.7.1.tgz

!./kafka_2.12-3.7.1/bin/zookeeper-server-start.sh -daemon ./kafka_2.12-3.7.1/config/zookeeper.properties
!./kafka_2.12-3.7.1/bin/kafka-server-start.sh -daemon ./kafka_2.12-3.7.1/config/server.properties
!echo "Waiting for 10 secs until kafka and zookeeper services are up and running"
!sleep 10
!ps -ef | grep kafka

In [ ]:
# Install required packages
!pip install kafka-python
!pip install dash
!pip install dash-bootstrap-components
!pip install plotly
!pip install confluent-kafka

In [ ]:
from sklearn.linear_model import SGDRegressor
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import functions as F
from pyspark.sql.window import Window

df_processed = df.drop("vicprice", "vicdemand", "transfer")

# Create a running average column based on 'vicdemand'
window_spec = Window.orderBy("day").rowsBetween(-10, 0)  # Adjust window size as needed
df_processed = df_processed.withColumn("running_avg_nswdemand", F.avg("nswdemand").over(window_spec))
window_spec_next = Window.orderBy("day")
df_processed = df_processed.withColumn("next_day_nswdemand", F.lead("nswdemand", 1).over(window_spec_next))
avg_nswdemand = df_processed.select(F.avg("nswdemand")).first()[0]
df_processed = df_processed.withColumn(
    "next_day_nswdemand",
    F.when(F.col("next_day_nswdemand").isNull(), avg_nswdemand).otherwise(F.col("next_day_nswdemand"))
)
noise_magnitude = 0.05 * avg_nswdemand
df_processed = df_processed.withColumn(
    "nswdemand",
    F.col("nswdemand") + (F.rand(seed=42) - 0.5) * 2 * noise_magnitude
)
df_processed.show()

+-------+---+--------+--------+-------------------+---------------------+------------------+
|   date|day|  period|nswprice|          nswdemand|running_avg_nswdemand|next_day_nswdemand|
+-------+---+--------+--------+-------------------+---------------------+------------------+
|2.65E-4|  1|     0.0|0.047106|0.28772452910185575|             0.282654|          0.247694|
|2.65E-4|  1|0.021277|0.047106| 0.2481024813385884|             0.265174|          0.233859|
|2.65E-4|  1|0.042553|0.047106|0.24800524850405736|   0.2547356666666667|          0.185808|
|2.65E-4|  1| 0.06383|0.046956|0.17573529925663744|           0.23750375|          0.154865|
|2.65E-4|  1|0.085106|0.046956|0.16210930391250145|             0.220976|          0.127789|
|2.65E-4|  1|0.106383|0.046956| 0.1285261792133506|  0.20544483333333333|          0.112169|
|2.65E-4|  1| 0.12766|0.046956|0.13340348600026813|  0.19211971428571428|          0.101309|
|2.65E-4|  1|0.148936|0.046806|0.08301315201076573|          0.1807683

In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import json
import threading
import time
from datetime import datetime
from kafka import KafkaProducer, KafkaConsumer
import tensorflow_io as tfio
import numpy as np
from scipy.stats import wasserstein_distance, ks_2samp
import random

pre_trained_cnt, pre_trained_sum = 1,0
online_cnt, online_sum = 1,0

KAFKA_TOPIC = 'electricity_demand'
KAFKA_BOOTSTRAP_SERVERS = 'localhost:9092'

from pyspark.ml.linalg import Vectors
from pyspark.sql import Row
import time
import numpy as np
import pandas as pd
from kafka import KafkaProducer
import json

class KafkaDataProducer:
    def __init__(self, spark_df, pre_trained_model, online_model):
        self.producer = KafkaProducer(
            bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
            value_serializer=lambda x: json.dumps(x).encode('utf-8')
        )

        self.data = spark_df.toPandas()
        self.current_index = 0
        self.pre_trained_model = pre_trained_model
        self.online_model = online_model

    def start_producing(self):
        def produce_data():
            global pre_trained_cnt, pre_trained_sum, online_cnt, online_sum
            while self.current_index < len(self.data):
                row = self.data.iloc[self.current_index]
                last_row = row
                if self.current_index != 0:
                    last_row = self.data.iloc[self.current_index - 1]

                features_vector = Vectors.dense([float(last_row['nswdemand']),float(last_row['running_avg_nswdemand']), int(row['day']), float(row['period'])])

                pandas_df = pd.DataFrame({'features': [features_vector], 'next_day_nswdemand': [float(row['nswdemand'])]})

                spark_df = spark.createDataFrame(pandas_df)

                batch_X = spark_df.select("features").rdd.map(lambda row: row['features'].toArray()).collect()
                pre_trained_prediction = self.pre_trained_model.predict(batch_X)[0]

                online_prediction = self.online_model.predict(batch_X)[0]

                batch_X = spark_df.select("features").rdd.map(lambda row: row['features'].toArray()).collect()
                batch_y = spark_df.select("next_day_nswdemand").rdd.map(lambda row: row['next_day_nswdemand']).collect()
                self.online_model.partial_fit(batch_X, batch_y)
                if self.current_index != 0:
                    pre_trained_cnt += 1
                    online_cnt += 1
                    pre_trained_sum += abs(float(last_row['nswdemand']) - float(pre_trained_prediction))
                    online_sum += abs(float(last_row['nswdemand']) - float(online_prediction))

                message = {
                    'timestamp': self.current_index,
                    'nswdemand': float(row['nswdemand']),
                    'pre_trained_prediction': float(pre_trained_prediction),
                    'online_prediction': float(online_prediction)
                }

                self.producer.send(KAFKA_TOPIC, message)
                self.current_index += 1
                time.sleep(0.05 + random.randint(1,10) / 1000)

            self.producer.flush()
            self.current_index = 0

        thread = threading.Thread(target=produce_data)
        thread.daemon = True
        thread.start()

def predict(model, df_batch):
    predictions = model.transform(df_batch)
    return predictions.select("next_day_nswdemand", "prediction")

class RealtimeDashboard:
    def __init__(self, pre_trained_model, online_model):

        self.x_data = []
        self.y_data = []
        self.y_pre_trained_pred = []
        self.y_online_pred = []
        self.predicted_demand = []
        self.abs_diff_sum_pre_trained = 0
        self.abs_diff_sum_online = 0
        self.DISPLAY_WINDOW = 200
        self.DRIFT_WINDOW = 20
        self.DRIFT_THRESHOLD = 2.0
        self.KS_PVALUE_THRESHOLD = 0.2
        self.WASSERSTEIN_THRESHOLD = 0.2

        self.drift_points = []
        self.drift_points_ks = []
        self.drift_points_wDistance = []
        self.y_pre_trained_pe = []
        self.y_online_pe = []

        self.consumer = KafkaConsumer(
            KAFKA_TOPIC,
            bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
            value_deserializer=lambda x: json.loads(x.decode('utf-8')),
            auto_offset_reset='earliest'
        )

        self.app = dash.Dash(__name__)
        self.create_layout()
        self.setup_callbacks()

        self.pre_trained_model = pre_trained_model
        self.online_model = online_model

        self.start_consuming()

    def start_consuming(self):
        def consume_data():
            previous_nswdemand = None
            for message in self.consumer:
                data = message.value
                current_nswdemand = data['nswdemand']

                # Calculate Percentage Error using the previous demand and current predictions
                if previous_nswdemand is not None:
                    pre_trained_pe = abs((previous_nswdemand - data['pre_trained_prediction']) / previous_nswdemand) * 100
                    online_pe = abs((previous_nswdemand - data['online_prediction']) / previous_nswdemand) * 100

                    self.y_pre_trained_pe.append(pre_trained_pe)
                    self.y_online_pe.append(online_pe)

                self.x_data.append(data['timestamp'])
                self.y_data.append(current_nswdemand)
                self.y_pre_trained_pred.append(data['pre_trained_prediction'])
                self.y_online_pred.append(data['online_prediction'])

                self.detect_drift()
                self.detect_drift_ks()
                self.detect_drift_wDistance()

                # Use the previous demand for deviation calculations
                if previous_nswdemand is not None:
                    self.abs_diff_sum_pre_trained += abs(previous_nswdemand - data['pre_trained_prediction'])
                    self.abs_diff_sum_online += abs(previous_nswdemand - data['online_prediction'])

                if len(self.x_data) > self.DISPLAY_WINDOW:
                    self.x_data = self.x_data[-self.DISPLAY_WINDOW:]
                    self.y_data = self.y_data[-self.DISPLAY_WINDOW:]
                    self.y_pre_trained_pred = self.y_pre_trained_pred[-self.DISPLAY_WINDOW:]
                    self.y_online_pred = self.y_online_pred[-self.DISPLAY_WINDOW:]
                    self.y_pre_trained_pe = self.y_pre_trained_pe[-self.DISPLAY_WINDOW:]
                    self.y_online_pe = self.y_online_pe[-self.DISPLAY_WINDOW:]

                previous_nswdemand = current_nswdemand

        thread = threading.Thread(target=consume_data)
        thread.daemon = True
        thread.start()


    def create_layout(self):

        self.app.layout = html.Div([
            html.H1("Real-time Electricity Demand Dashboard",
                    style={'textAlign': 'center', 'color': '#2c3e50', 'marginBottom': 20}),

            html.Div([
                dcc.Graph(id='live-graph', animate=True),
                dcc.Graph(id='stats-graph', animate=True),
            ], style={'display': 'flex', 'flexDirection': 'row'}),

            html.Div([
                dcc.Graph(id='demand-with-predictions', animate=True),
            ], style={'marginTop': 20}),
            html.Div([
                dcc.Graph(id='percentage-error-graph', animate=True),
            ], style={'marginTop': 20}),
            html.Div([
                html.P(id='pre-trained-deviation'),
                html.P(id='online-deviation'),
            ]),

            dcc.Interval(
                id='graph-update',
                interval=1000,
                n_intervals=0
            )
        ])

    def detect_drift_ks(self):
        if len(self.y_data) >= self.DRIFT_WINDOW * 2:

            prev_window = self.y_data[-2*self.DRIFT_WINDOW:-self.DRIFT_WINDOW]
            curr_window = self.y_data[-self.DRIFT_WINDOW:]

            ks_stat, ks_p_value = ks_2samp(prev_window, curr_window)

            drift_detected = False

            if ks_p_value < self.KS_PVALUE_THRESHOLD:
                drift_detected = True

            if drift_detected:

                if len(self.drift_points_ks) == 0 or abs(self.drift_points_ks[-1] - self.x_data[-1]) >= 20:
                    self.drift_points_ks.append(self.x_data[-1])
                    return True

        return False
    def detect_drift_wDistance(self):
        if len(self.y_data) >= self.DRIFT_WINDOW * 2:

            prev_window = self.y_data[-2*self.DRIFT_WINDOW:-self.DRIFT_WINDOW]
            curr_window = self.y_data[-self.DRIFT_WINDOW:]

            wasserstein_dist = wasserstein_distance(prev_window, curr_window)

            drift_detected = False

            if wasserstein_dist > self.WASSERSTEIN_THRESHOLD:
                drift_detected = True

            if drift_detected:

                if len(self.drift_points_wDistance) == 0 or abs(self.drift_points_wDistance[-1] - self.x_data[-1]) >= 20:
                    self.drift_points_wDistance.append(self.x_data[-1])
                    return True

        return False

    def detect_drift(self):
        if len(self.y_data) >= self.DRIFT_WINDOW * 2:
            prev_window = self.y_data[-2*self.DRIFT_WINDOW:-self.DRIFT_WINDOW]
            curr_window = self.y_data[-self.DRIFT_WINDOW:]

            wasserstein_dist = wasserstein_distance(prev_window, curr_window)
            ks_stat, ks_p_value = ks_2samp(prev_window, curr_window)

            drift_detected = False
            if wasserstein_dist > self.WASSERSTEIN_THRESHOLD:
                drift_detected = True
            if ks_p_value < self.KS_PVALUE_THRESHOLD:
                drift_detected = True

            if drift_detected:
                if len(self.drift_points) == 0 or abs(self.drift_points[-1] - self.x_data[-1]) >= 20:
                    self.drift_points.append(self.x_data[-1])
                    return True

        return False

    def calculate_statistics(self):
        if len(self.y_data) >= 10:
            window_size = 10
            rolling_mean = pd.Series(self.y_data).rolling(window=window_size).mean()
            rolling_std = pd.Series(self.y_data).rolling(window=window_size).std()
            return rolling_mean.iloc[-self.DISPLAY_WINDOW:], rolling_std.iloc[-self.DISPLAY_WINDOW:]
        return None, None

    def setup_callbacks(self):
        @self.app.callback(
            [Output('live-graph', 'figure'),
             Output('stats-graph', 'figure'),
             Output('demand-with-predictions', 'figure'),
             Output('percentage-error-graph', 'figure'),
             Output('pre-trained-deviation', 'children'),
             Output('online-deviation', 'children')],
            [Input('graph-update', 'n_intervals')]
        )
        def update_graphs(n):

            if not self.x_data or not self.y_data:
                return go.Figure(), go.Figure(), go.Figure()

            main_fig = go.Figure()

            main_fig.add_trace(go.Scatter(
                x=self.x_data[-self.DISPLAY_WINDOW:],
                y=self.y_data[-self.DISPLAY_WINDOW:],
                mode='lines+markers',
                name='Demand',
                line=dict(color='#2980b9')
            ))

            y_min = min(self.y_data[-self.DISPLAY_WINDOW:]) if self.y_data else 0
            y_max = max(self.y_data[-self.DISPLAY_WINDOW:]) if self.y_data else 1

            for drift_point in self.drift_points:
                if drift_point >= min(self.x_data[-self.DISPLAY_WINDOW:]) and \
                   drift_point <= max(self.x_data[-self.DISPLAY_WINDOW:]):

                    line_color = "purple" if drift_point in self.drift_points_ks and drift_point in self.drift_points_wDistance else \
                                "blue" if drift_point in self.drift_points_ks else \
                                "green" if drift_point in self.drift_points_wDistance else \
                                "red"
                    line_dash = "dashdot" if drift_point in self.drift_points_ks and drift_point in self.drift_points_wDistance else \
                               "dot" if drift_point in self.drift_points_ks else \
                               "solid" if drift_point in self.drift_points_wDistance else \
                               "dash"

                    main_fig.add_shape(
                        type="line",
                        x0=drift_point,
                        x1=drift_point,
                        y0=y_min,
                        y1=y_max,
                        line=dict(
                            color=line_color,
                            width=2,
                            dash=line_dash,
                        )
                    )

            main_fig.update_layout(
                title='Real-time Electricity Demand with Drift Detection',
                xaxis_title='Time',
                yaxis_title='Demand (MW)',
                plot_bgcolor='white',
                paper_bgcolor='white',
                showlegend=True,
                yaxis=dict(
                    gridcolor='#f0f0f0',
                    range=[min(self.y_data[-50:]) * 0.95,
                          max(self.y_data[-50:]) * 1.05]
                ),
                xaxis=dict(
                    gridcolor='#f0f0f0',
                    range=[min(self.x_data[-50:]),
                          max(self.x_data[-50:])]
                )
            )

            stats_fig = go.Figure()
            rolling_mean, rolling_std = self.calculate_statistics()

            if rolling_mean is not None and rolling_std is not None:
                stats_fig.add_trace(go.Scatter(
                    x=self.x_data[-self.DISPLAY_WINDOW:],
                    y=rolling_mean,
                    mode='lines',
                    name='Moving Average',
                    line=dict(color='#27ae60')
                ))

                stats_fig.add_trace(go.Scatter(
                    x=self.x_data[-self.DISPLAY_WINDOW:],
                    y=rolling_mean + rolling_std,
                    mode='lines',
                    name='Upper Bound',
                    line=dict(color='#e74c3c', dash='dash')
                ))

                stats_fig.add_trace(go.Scatter(
                    x=self.x_data[-self.DISPLAY_WINDOW:],
                    y=rolling_mean - rolling_std,
                    mode='lines',
                    name='Lower Bound',
                    line=dict(color='#e74c3c', dash='dash'),
                    fill='tonexty'
                ))

            stats_fig.update_layout(
                title='Moving Average and Volatility',
                xaxis_title='Time',
                yaxis_title='Demand (MW)',
                plot_bgcolor='white',
                paper_bgcolor='white',
                showlegend=True,
                yaxis=dict(
                    gridcolor='#f0f0f0',
                    range=[min(self.y_data[-50:]) * 0.95,
                          max(self.y_data[-50:]) * 1.05]
                ),
                xaxis=dict(
                    gridcolor='#f0f0f0',
                    range=[min(self.x_data[-50:]),
                          max(self.x_data[-50:])]
                )
            )

            predictions_fig = go.Figure()

            predictions_fig.add_trace(go.Scatter(
                x=self.x_data[-self.DISPLAY_WINDOW:],
                y=self.y_data[-self.DISPLAY_WINDOW:],
                mode='lines',
                name='Actual Demand',
                line=dict(color='#3498db')
            ))

            if self.y_pre_trained_pred:
                predictions_fig.add_trace(go.Scatter(
                    x=[_ - 1 for _ in self.x_data[-self.DISPLAY_WINDOW:]],
                    y=self.y_pre_trained_pred[-self.DISPLAY_WINDOW:],
                    mode='lines',
                    name='Predicted Demand for Pre Trained Model',
                    line=dict(color='#e6ee12', dash='dash')
                ))
            if self.y_online_pred:
                predictions_fig.add_trace(go.Scatter(
                    x=[_ - 1 for _ in self.x_data[-self.DISPLAY_WINDOW:]],
                    y=self.y_online_pred[-self.DISPLAY_WINDOW:],
                    mode='lines',
                    name='Predicted Demand for Online Learning Model',
                    line=dict(color='#e67e22', dash='dash')
                ))

            predictions_fig.update_layout(
                title='Electricity Demand and Predictions',
                xaxis_title='Time',
                yaxis_title='Demand (MW)',
                plot_bgcolor='white',
                paper_bgcolor='white',
                showlegend=True,
                yaxis=dict(
                    gridcolor='#f0f0f0',
                    range=[min(self.y_data[-50:]) * 0.95,
                          max(self.y_data[-50:]) * 1.05]
                ),
                xaxis=dict(
                    gridcolor='#f0f0f0',
                    range=[min(self.x_data[-50:]),
                          max(self.x_data[-50:])]
                )
            )
            global pre_trained_sum, pre_trained_cnt, online_sum, online_cnt
            pre_train = (pre_trained_sum / pre_trained_cnt)
            online_train = (online_sum / online_cnt)
            pe_fig = go.Figure()

            if self.y_pre_trained_pe:
                pe_fig.add_trace(go.Scatter(
                    x=self.x_data[-self.DISPLAY_WINDOW:],
                    y=self.y_pre_trained_pe[-self.DISPLAY_WINDOW:],
                    mode='lines',
                    name='Pre-trained Model % Error',
                    line=dict(color='#e6ee12', dash='dash')
                ))

            if self.y_online_pe:
                pe_fig.add_trace(go.Scatter(
                    x=self.x_data[-self.DISPLAY_WINDOW:],
                    y=self.y_online_pe[-self.DISPLAY_WINDOW:],
                    mode='lines',
                    name='Online Learning Model % Error',
                    line=dict(color='#e67e22', dash='dash')
                ))

            pe_fig.update_layout(
                title='Percentage Error Comparison',
                xaxis_title='Time',
                yaxis_title='Percentage Error (%)',
                plot_bgcolor='white',
                paper_bgcolor='white',
                showlegend=True,
                yaxis=dict(
                    gridcolor='#f0f0f0',
                    range=[0, 20]
                ),
                xaxis=dict(
                    gridcolor='#f0f0f0',
                    range=[min(self.x_data[-self.DISPLAY_WINDOW:]),
                           max(self.x_data[-self.DISPLAY_WINDOW:])]
                )
            )

            return main_fig, stats_fig, predictions_fig, pe_fig, \
                   f"Pre-trained Model Average Deviation: {pre_train:.5f}", \
                   f"Online Model Average Deviation: {online_train:.5f}"

    def run_server(self, debug=True, port=8052):
        self.app.run_server(debug=debug, port=port)

def main(spark_df):
    assembler = VectorAssembler(inputCols=["nswdemand","running_avg_nswdemand","day","period"], outputCol="features")
    df_features = assembler.transform(df_processed.drop("vicprice", "vicdemand", "transfer")).select("features", "next_day_nswdemand")

    online_model = SGDRegressor(loss = 'squared_error')
    lr = SGDRegressor(loss = 'squared_error', alpha=0.005)
    batch_X = df_features.select("features").rdd.map(lambda row: row['features'].toArray()).collect()
    batch_y = df_features.select("next_day_nswdemand").rdd.map(lambda row: row['next_day_nswdemand']).collect()
    online_model.fit(batch_X, batch_y)
    lr.fit(batch_X,batch_y)
    pre_trained_model = lr

    producer = KafkaDataProducer(spark_df, pre_trained_model, online_model)
    producer.start_producing()

    dashboard = RealtimeDashboard(pre_trained_model, online_model)
    dashboard.run_server()

if __name__ == "__main__":
    main(df_processed)

<IPython.core.display.Javascript object>

In [ ]:
from sklearn.linear_model import SGDRegressor
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import functions as F
from pyspark.sql.window import Window

df_processed = df.drop("vicprice")

# Create a running average column based on 'vicdemand'
window_spec = Window.orderBy("day").rowsBetween(-10, 0)  # Adjust window size as needed
df_processed = df_processed.withColumn("running_avg_nswdemand", F.avg("nswdemand").over(window_spec))
df_processed = df_processed.withColumn("running_avg_vicdemand", F.avg("vicdemand").over(window_spec))
window_spec_next = Window.orderBy("day")
df_processed = df_processed.withColumn("next_day_nswdemand", F.lead("nswdemand", 1).over(window_spec_next))
df_processed = df_processed.withColumn("next_day_vicdemand", F.lead("vicdemand", 1).over(window_spec_next))
avg_nswdemand = df_processed.select(F.avg("nswdemand")).first()[0]
df_processed = df_processed.withColumn(
    "next_day_nswdemand",
    F.when(F.col("next_day_nswdemand").isNull(), avg_nswdemand).otherwise(F.col("next_day_nswdemand"))
)
noise_magnitude = 0.05 * avg_nswdemand
df_processed = df_processed.withColumn(
    "nswdemand",
    F.col("nswdemand") + (F.rand(seed=42) - 0.5) * 2 * noise_magnitude
)
avg_vicdemand = df_processed.select(F.avg("vicdemand")).first()[0]
df_processed = df_processed.withColumn(
    "next_day_vicdemand",
    F.when(F.col("next_day_vicdemand").isNull(), avg_vicdemand).otherwise(F.col("next_day_vicdemand"))
)
noise_magnitude_vic = 0.05 * avg_vicdemand
df_processed = df_processed.withColumn(
    "vicdemand",
    F.col("vicdemand") + (F.rand(seed=42) - 0.5) * 2 * noise_magnitude_vic
)
df_processed.show()

In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import json
import threading
import time
from datetime import datetime
from kafka import KafkaProducer, KafkaConsumer
import tensorflow_io as tfio
import numpy as np
from scipy.stats import wasserstein_distance, ks_2samp
import random

pre_trained_cnt_nsw, pre_trained_sum_nsw = 1, 0
online_cnt_nsw, online_sum_nsw = 1, 0
pre_trained_cnt_vic, pre_trained_sum_vic = 1, 0
online_cnt_vic, online_sum_vic = 1, 0

KAFKA_TOPIC = 'electricity_demand'
KAFKA_BOOTSTRAP_SERVERS = 'localhost:9092'

from pyspark.ml.linalg import Vectors
from pyspark.sql import Row
import time
import numpy as np
import pandas as pd
from kafka import KafkaProducer
import json

class KafkaDataProducer:
    def __init__(self, spark_df, pre_trained_model_nsw, online_model_nsw,
                 pre_trained_model_vic, online_model_vic):
        self.producer = KafkaProducer(
            bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
            value_serializer=lambda x: json.dumps(x).encode('utf-8')
        )

        self.data = spark_df.toPandas()
        self.current_index = 0
        self.pre_trained_model_nsw = pre_trained_model_nsw
        self.online_model_nsw = online_model_nsw
        self.pre_trained_model_vic = pre_trained_model_vic
        self.online_model_vic = online_model_vic

    def start_producing(self):
        def produce_data():
            global pre_trained_cnt_nsw, pre_trained_sum_nsw, online_cnt_nsw, online_sum_nsw, \
                   pre_trained_cnt_vic, pre_trained_sum_vic, online_cnt_vic, online_sum_vic
            while self.current_index < len(self.data):
                row = self.data.iloc[self.current_index]
                last_row = row
                if self.current_index != 0:
                    last_row = self.data.iloc[self.current_index - 1]

                # NSWDemand Features and Prediction
                nsw_features_vector = Vectors.dense([
                    float(last_row['nswdemand']),
                    float(last_row['running_avg_nswdemand']),
                    int(row['day']),
                    float(row['period'])
                ])

                # VicDemand Features and Prediction
                vic_features_vector = Vectors.dense([
                    float(last_row['vicdemand']),
                    float(last_row['running_avg_vicdemand']),
                    int(row['day']),
                    float(row['period'])
                ])

                # Prepare Pandas DataFrames
                nsw_pandas_df = pd.DataFrame({
                    'features': [nsw_features_vector],
                    'next_day_nswdemand': [float(row['nswdemand'])]
                })
                vic_pandas_df = pd.DataFrame({
                    'features': [vic_features_vector],
                    'next_day_vicdemand': [float(row['vicdemand'])]
                })

                # Convert to Spark DataFrames
                nsw_spark_df = spark.createDataFrame(nsw_pandas_df)
                vic_spark_df = spark.createDataFrame(vic_pandas_df)

                # Predict for both NSW and Victoria
                nsw_batch_X = nsw_spark_df.select("features").rdd.map(lambda row: row['features'].toArray()).collect()
                vic_batch_X = vic_spark_df.select("features").rdd.map(lambda row: row['features'].toArray()).collect()

                nsw_pre_trained_prediction = self.pre_trained_model_nsw.predict(nsw_batch_X)[0]
                nsw_online_prediction = self.online_model_nsw.predict(nsw_batch_X)[0]

                vic_pre_trained_prediction = self.pre_trained_model_vic.predict(vic_batch_X)[0]
                vic_online_prediction = self.online_model_vic.predict(vic_batch_X)[0]

                # Online model partial fit
                nsw_batch_y = nsw_spark_df.select("next_day_nswdemand").rdd.map(lambda row: row['next_day_nswdemand']).collect()
                vic_batch_y = vic_spark_df.select("next_day_vicdemand").rdd.map(lambda row: row['next_day_vicdemand']).collect()

                self.online_model_nsw.partial_fit(nsw_batch_X, nsw_batch_y)
                self.online_model_vic.partial_fit(vic_batch_X, vic_batch_y)

                # Track prediction errors (similar to before, but separate for each state)
                if self.current_index != 0:
                    # NSW tracking
                    pre_trained_cnt_nsw += 1
                    online_cnt_nsw += 1
                    pre_trained_sum_nsw += abs(float(last_row['nswdemand']) - float(nsw_pre_trained_prediction))
                    online_sum_nsw += abs(float(last_row['nswdemand']) - float(nsw_online_prediction))

                    # Victoria tracking
                    pre_trained_cnt_vic += 1
                    online_cnt_vic += 1
                    pre_trained_sum_vic += abs(float(last_row['vicdemand']) - float(vic_pre_trained_prediction))
                    online_sum_vic += abs(float(last_row['vicdemand']) - float(vic_online_prediction))

                # Prepare Kafka message with both cities' data
                message = {
                    'timestamp': self.current_index,
                    'nsw_demand': float(row['nswdemand']),
                    'nsw_pre_trained_prediction': float(nsw_pre_trained_prediction),
                    'nsw_online_prediction': float(nsw_online_prediction),
                    'vic_demand': float(row['vicdemand']),
                    'vic_pre_trained_prediction': float(vic_pre_trained_prediction),
                    'vic_online_prediction': float(vic_online_prediction)
                }

                self.producer.send(KAFKA_TOPIC, message)
                self.current_index += 1
                time.sleep(0.05 + random.randint(1,10) / 1000)

            self.producer.flush()
            self.current_index = 0

        thread = threading.Thread(target=produce_data)
        thread.daemon = True
        thread.start()

def predict(model, df_batch):
    predictions = model.transform(df_batch)
    return predictions.select("next_day_nswdemand", "prediction")

class RealtimeDashboard:
    def __init__(self, pre_trained_models, online_models):

        self.x_data = {'NSW': [], 'VIC': []}
        self.y_data = {'NSW': [], 'VIC': []}
        self.y_pre_trained_pred = {'NSW': [], 'VIC': []}
        self.y_online_pred = {'NSW': [], 'VIC': []}
        self.y_pre_trained_pe = {'NSW': [], 'VIC': []}
        self.y_online_pe = {'NSW': [], 'VIC': []}
        self.predicted_demand = []
        self.abs_diff_sum_pre_trained = 0
        self.abs_diff_sum_online = 0
        self.DISPLAY_WINDOW = 200
        self.DRIFT_WINDOW = 20
        self.DRIFT_THRESHOLD = 2.0
        self.KS_PVALUE_THRESHOLD = 0.2
        self.WASSERSTEIN_THRESHOLD = 0.2

        self.drift_points = []
        self.drift_points_ks = []
        self.drift_points_wDistance = []

        self.consumer = KafkaConsumer(
            KAFKA_TOPIC,
            bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
            value_deserializer=lambda x: json.loads(x.decode('utf-8')),
            auto_offset_reset='earliest'
        )

        self.app = dash.Dash(__name__)
        self.create_layout()
        self.setup_callbacks()

        self.start_consuming()

    def start_consuming(self):
        def consume_data():
            previous_nswdemand = None
            for message in self.consumer:
                data = message.value

                # Process NSW data
                self.x_data['NSW'].append(data['timestamp'])
                self.y_data['NSW'].append(data['nsw_demand'])
                self.y_pre_trained_pred['NSW'].append(data['nsw_pre_trained_prediction'])
                self.y_online_pred['NSW'].append(data['nsw_online_prediction'])

                # Process Victoria data
                self.x_data['VIC'].append(data['timestamp'])
                self.y_data['VIC'].append(data['vic_demand'])
                self.y_pre_trained_pred['VIC'].append(data['vic_pre_trained_prediction'])
                self.y_online_pred['VIC'].append(data['vic_online_prediction'])

                # Calculate Percentage Error for both cities
                if len(self.y_data['NSW']) > 1:
                    nsw_pre_trained_pe = abs((self.y_data['NSW'][-2] - data['nsw_pre_trained_prediction']) / self.y_data['NSW'][-2]) * 100
                    nsw_online_pe = abs((self.y_data['NSW'][-2] - data['nsw_online_prediction']) / self.y_data['NSW'][-2]) * 100
                    self.y_pre_trained_pe['NSW'].append(nsw_pre_trained_pe)
                    self.y_online_pe['NSW'].append(nsw_online_pe)

                if len(self.y_data['VIC']) > 1:
                    vic_pre_trained_pe = abs((self.y_data['VIC'][-2] - data['vic_pre_trained_prediction']) / self.y_data['VIC'][-2]) * 100
                    vic_online_pe = abs((self.y_data['VIC'][-2] - data['vic_online_prediction']) / self.y_data['VIC'][-2]) * 100
                    self.y_pre_trained_pe['VIC'].append(vic_pre_trained_pe)
                    self.y_online_pe['VIC'].append(vic_online_pe)

                # Trim data to display window for both cities
                for city in ['NSW', 'VIC']:
                    if len(self.x_data[city]) > self.DISPLAY_WINDOW:
                        self.x_data[city] = self.x_data[city][-self.DISPLAY_WINDOW:]
                        self.y_data[city] = self.y_data[city][-self.DISPLAY_WINDOW:]
                        self.y_pre_trained_pred[city] = self.y_pre_trained_pred[city][-self.DISPLAY_WINDOW:]
                        self.y_online_pred[city] = self.y_online_pred[city][-self.DISPLAY_WINDOW:]
                        self.y_pre_trained_pe[city] = self.y_pre_trained_pe[city][-self.DISPLAY_WINDOW:]
                        self.y_online_pe[city] = self.y_online_pe[city][-self.DISPLAY_WINDOW:]

        thread = threading.Thread(target=consume_data)
        thread.daemon = True
        thread.start()


    def create_layout(self):

        self.app.layout = html.Div([
            html.H1("Real-time Electricity Demand Dashboard",
                    style={'textAlign': 'center', 'color': '#2c3e50', 'marginBottom': 20}),

            # Add dropdown for city selection
            dcc.Dropdown(
                id='city-dropdown',
                options=[
                    {'label': 'New South Wales', 'value': 'NSW'},
                    {'label': 'Victoria', 'value': 'VIC'}
                ],
                value='NSW',  # Default to NSW
                style={'width': '50%', 'margin': '0 auto'}
            ),
            html.H1("Real-time Electricity Demand Dashboard",
                    style={'textAlign': 'center', 'color': '#2c3e50', 'marginBottom': 20}),

            html.Div([
                dcc.Graph(id='live-graph', animate=True),
                dcc.Graph(id='stats-graph', animate=True),
            ], style={'display': 'flex', 'flexDirection': 'row'}),

            html.Div([
                dcc.Graph(id='demand-with-predictions', animate=True),
            ], style={'marginTop': 20}),
            html.Div([
                dcc.Graph(id='percentage-error-graph', animate=True),
            ], style={'marginTop': 20}),
            html.Div([
                html.P(id='pre-trained-deviation'),
                html.P(id='online-deviation'),
            ]),

            dcc.Interval(
                id='graph-update',
                interval=1000,
                n_intervals=0
            )
        ])

    def detect_drift_ks(self):
        if len(self.y_data) >= self.DRIFT_WINDOW * 2:

            prev_window = self.y_data[-2*self.DRIFT_WINDOW:-self.DRIFT_WINDOW]
            curr_window = self.y_data[-self.DRIFT_WINDOW:]

            ks_stat, ks_p_value = ks_2samp(prev_window, curr_window)

            drift_detected = False

            if ks_p_value < self.KS_PVALUE_THRESHOLD:
                drift_detected = True

            if drift_detected:

                if len(self.drift_points_ks) == 0 or abs(self.drift_points_ks[-1] - self.x_data[-1]) >= 20:
                    self.drift_points_ks.append(self.x_data[-1])
                    return True

        return False
    def detect_drift_wDistance(self):
        if len(self.y_data) >= self.DRIFT_WINDOW * 2:

            prev_window = self.y_data[-2*self.DRIFT_WINDOW:-self.DRIFT_WINDOW]
            curr_window = self.y_data[-self.DRIFT_WINDOW:]

            wasserstein_dist = wasserstein_distance(prev_window, curr_window)

            drift_detected = False

            if wasserstein_dist > self.WASSERSTEIN_THRESHOLD:
                drift_detected = True

            if drift_detected:

                if len(self.drift_points_wDistance) == 0 or abs(self.drift_points_wDistance[-1] - self.x_data[-1]) >= 20:
                    self.drift_points_wDistance.append(self.x_data[-1])
                    return True

        return False

    def detect_drift(self):
        if len(self.y_data) >= self.DRIFT_WINDOW * 2:
            prev_window = self.y_data[-2*self.DRIFT_WINDOW:-self.DRIFT_WINDOW]
            curr_window = self.y_data[-self.DRIFT_WINDOW:]

            wasserstein_dist = wasserstein_distance(prev_window, curr_window)
            ks_stat, ks_p_value = ks_2samp(prev_window, curr_window)

            drift_detected = False
            if wasserstein_dist > self.WASSERSTEIN_THRESHOLD:
                drift_detected = True
            if ks_p_value < self.KS_PVALUE_THRESHOLD:
                drift_detected = True

            if drift_detected:
                if len(self.drift_points) == 0 or abs(self.drift_points[-1] - self.x_data[-1]) >= 20:
                    self.drift_points.append(self.x_data[-1])
                    return True

        return False

    def calculate_statistics(self,y_data):
        if len(y_data) >= 10:
            window_size = 10
            rolling_mean = pd.Series(y_data).rolling(window=window_size).mean()
            rolling_std = pd.Series(y_data).rolling(window=window_size).std()
            return rolling_mean.iloc[-self.DISPLAY_WINDOW:], rolling_std.iloc[-self.DISPLAY_WINDOW:]
        return None, None

    def setup_callbacks(self):
        @self.app.callback(
            [Output('live-graph', 'figure'),
             Output('stats-graph', 'figure'),
             Output('demand-with-predictions', 'figure'),
             Output('percentage-error-graph', 'figure'),
             Output('pre-trained-deviation', 'children'),
             Output('online-deviation', 'children')],
            [Input('city-dropdown', 'value'),Input('graph-update', 'n_intervals')]
        )
        def update_graphs(selected_city, n):
            x_data = self.x_data[selected_city][-self.DISPLAY_WINDOW:]
            y_data = self.y_data[selected_city][-self.DISPLAY_WINDOW:]
            y_pre_trained_pred = self.y_pre_trained_pred[selected_city][-self.DISPLAY_WINDOW:]
            y_online_pred = self.y_online_pred[selected_city][-self.DISPLAY_WINDOW:]
            y_pre_trained_pe = self.y_pre_trained_pe[selected_city][-self.DISPLAY_WINDOW:]
            y_online_pe = self.y_online_pe[selected_city][-self.DISPLAY_WINDOW:]

            if not x_data or not y_data:
                return go.Figure(), go.Figure(), go.Figure(),go.Figure(),'',''

            main_fig = go.Figure()

            main_fig.add_trace(go.Scatter(
                x=x_data[-self.DISPLAY_WINDOW:],
                y=y_data[-self.DISPLAY_WINDOW:],
                mode='lines+markers',
                name='Demand',
                line=dict(color='#2980b9')
            ))

            y_min = min(y_data[-self.DISPLAY_WINDOW:]) if y_data else 0
            y_max = max(y_data[-self.DISPLAY_WINDOW:]) if y_data else 1

            for drift_point in self.drift_points:
                if drift_point >= min(x_data[-self.DISPLAY_WINDOW:]) and \
                   drift_point <= max(x_data[-self.DISPLAY_WINDOW:]):

                    line_color = "purple" if drift_point in self.drift_points_ks and drift_point in self.drift_points_wDistance else \
                                "blue" if drift_point in self.drift_points_ks else \
                                "green" if drift_point in self.drift_points_wDistance else \
                                "red"
                    line_dash = "dashdot" if drift_point in self.drift_points_ks and drift_point in self.drift_points_wDistance else \
                               "dot" if drift_point in self.drift_points_ks else \
                               "solid" if drift_point in self.drift_points_wDistance else \
                               "dash"

                    main_fig.add_shape(
                        type="line",
                        x0=drift_point,
                        x1=drift_point,
                        y0=y_min,
                        y1=y_max,
                        line=dict(
                            color=line_color,
                            width=2,
                            dash=line_dash,
                        )
                    )

            main_fig.update_layout(
                title='Real-time Electricity Demand with Drift Detection',
                xaxis_title='Time',
                yaxis_title='Demand (MW)',
                plot_bgcolor='white',
                paper_bgcolor='white',
                showlegend=True,
                yaxis=dict(
                    gridcolor='#f0f0f0',
                    range=[min(y_data[-50:]) * 0.95,
                          max(y_data[-50:]) * 1.05]
                ),
                xaxis=dict(
                    gridcolor='#f0f0f0',
                    range=[min(x_data[-50:]),
                          max(x_data[-50:])]
                )
            )

            stats_fig = go.Figure()
            rolling_mean, rolling_std = self.calculate_statistics(y_data)

            if rolling_mean is not None and rolling_std is not None:
                stats_fig.add_trace(go.Scatter(
                    x=x_data[-self.DISPLAY_WINDOW:],
                    y=rolling_mean,
                    mode='lines',
                    name='Moving Average',
                    line=dict(color='#27ae60')
                ))

                stats_fig.add_trace(go.Scatter(
                    x=x_data[-self.DISPLAY_WINDOW:],
                    y=rolling_mean + rolling_std,
                    mode='lines',
                    name='Upper Bound',
                    line=dict(color='#e74c3c', dash='dash')
                ))

                stats_fig.add_trace(go.Scatter(
                    x=x_data[-self.DISPLAY_WINDOW:],
                    y=rolling_mean - rolling_std,
                    mode='lines',
                    name='Lower Bound',
                    line=dict(color='#e74c3c', dash='dash'),
                    fill='tonexty'
                ))

            stats_fig.update_layout(
                title='Moving Average and Volatility',
                xaxis_title='Time',
                yaxis_title='Demand (MW)',
                plot_bgcolor='white',
                paper_bgcolor='white',
                showlegend=True,
                yaxis=dict(
                    gridcolor='#f0f0f0',
                    range=[min(y_data[-50:]) * 0.95,
                          max(y_data[-50:]) * 1.05]
                ),
                xaxis=dict(
                    gridcolor='#f0f0f0',
                    range=[min(x_data[-50:]),
                          max(x_data[-50:])]
                )
            )

            predictions_fig = go.Figure()

            predictions_fig.add_trace(go.Scatter(
                x=x_data[-self.DISPLAY_WINDOW:],
                y=y_data[-self.DISPLAY_WINDOW:],
                mode='lines',
                name='Actual Demand',
                line=dict(color='#3498db')
            ))

            if y_pre_trained_pred:
                predictions_fig.add_trace(go.Scatter(
                    x=[_ - 1 for _ in x_data[-self.DISPLAY_WINDOW:]],
                    y=y_pre_trained_pred[-self.DISPLAY_WINDOW:],
                    mode='lines',
                    name='Predicted Demand for Pre Trained Model',
                    line=dict(color='#e6ee12', dash='dash')
                ))
            if y_online_pred:
                predictions_fig.add_trace(go.Scatter(
                    x=[_ - 1 for _ in x_data[-self.DISPLAY_WINDOW:]],
                    y=y_online_pred[-self.DISPLAY_WINDOW:],
                    mode='lines',
                    name='Predicted Demand for Online Learning Model',
                    line=dict(color='#e67e22', dash='dash')
                ))

            predictions_fig.update_layout(
                title='Electricity Demand and Predictions',
                xaxis_title='Time',
                yaxis_title='Demand (MW)',
                plot_bgcolor='white',
                paper_bgcolor='white',
                showlegend=True,
                yaxis=dict(
                    gridcolor='#f0f0f0',
                    range=[min(y_data[-50:]) * 0.95,
                          max(y_data[-50:]) * 1.05]
                ),
                xaxis=dict(
                    gridcolor='#f0f0f0',
                    range=[min(x_data[-50:]),
                          max(x_data[-50:])]
                )
            )
            global pre_trained_sum_nsw, pre_trained_cnt_nsw, online_sum_nsw, online_cnt_nsw, \
               pre_trained_sum_vic, pre_trained_cnt_vic, online_sum_vic, online_cnt_vic

            if selected_city == 'NSW':
                pre_train = (pre_trained_sum_nsw / pre_trained_cnt_nsw)
                online_train = (online_sum_nsw / online_cnt_nsw)
            else:
                pre_train = (pre_trained_sum_vic / pre_trained_cnt_vic)
                online_train = (online_sum_vic / online_cnt_vic)

            pe_fig = go.Figure()

            if y_pre_trained_pe:
                pe_fig.add_trace(go.Scatter(
                    x=x_data[-self.DISPLAY_WINDOW:],
                    y=y_pre_trained_pe[-self.DISPLAY_WINDOW:],
                    mode='lines',
                    name='Pre-trained Model % Error',
                    line=dict(color='#e6ee12', dash='dash')
                ))

            if y_online_pe:
                pe_fig.add_trace(go.Scatter(
                    x=x_data[-self.DISPLAY_WINDOW:],
                    y=y_online_pe[-self.DISPLAY_WINDOW:],
                    mode='lines',
                    name='Online Learning Model % Error',
                    line=dict(color='#e67e22', dash='dash')
                ))

            pe_fig.update_layout(
                title='Percentage Error Comparison',
                xaxis_title='Time',
                yaxis_title='Percentage Error (%)',
                plot_bgcolor='white',
                paper_bgcolor='white',
                showlegend=True,
                yaxis=dict(
                    gridcolor='#f0f0f0',
                    range=[0, 20]
                ),
                xaxis=dict(
                    gridcolor='#f0f0f0',
                    range=[min(x_data[-self.DISPLAY_WINDOW:]),
                           max(x_data[-self.DISPLAY_WINDOW:])]
                )
            )

            return main_fig, stats_fig, predictions_fig, pe_fig, \
                   f"Pre-trained Model Average Deviation: {pre_train:.5f}", \
                   f"Online Model Average Deviation: {online_train:.5f}"

    def run_server(self, debug=True, port=8052):
        self.app.run_server(debug=debug, port=port)

def main(spark_df):
    assembler = VectorAssembler(inputCols=["nswdemand","running_avg_nswdemand","day","period"], outputCol="nsw_features")
    vic_assembler = VectorAssembler(inputCols=["vicdemand","running_avg_vicdemand","day","period"], outputCol="vic_features")

    # NSW Features
    nsw_df_features = assembler.transform(df_processed).select("nsw_features", "next_day_nswdemand")
    nsw_batch_X = nsw_df_features.select("nsw_features").rdd.map(lambda row: row['nsw_features'].toArray()).collect()
    nsw_batch_y = nsw_df_features.select("next_day_nswdemand").rdd.map(lambda row: row['next_day_nswdemand']).collect()

    # Victoria Features
    vic_df_features = vic_assembler.transform(df_processed).select("vic_features", "next_day_vicdemand")
    vic_batch_X = vic_df_features.select("vic_features").rdd.map(lambda row: row['vic_features'].toArray()).collect()
    vic_batch_y = vic_df_features.select("next_day_vicdemand").rdd.map(lambda row: row['next_day_vicdemand']).collect()

    # Create models for both states
    online_model_nsw = SGDRegressor(loss='squared_error')
    online_model_vic = SGDRegressor(loss='squared_error')

    lr_nsw = SGDRegressor(loss='squared_error', alpha=0.005)
    lr_vic = SGDRegressor(loss='squared_error', alpha=0.005)

    # Fit models
    online_model_nsw.fit(nsw_batch_X, nsw_batch_y)
    lr_nsw.fit(nsw_batch_X, nsw_batch_y)

    online_model_vic.fit(vic_batch_X, vic_batch_y)
    lr_vic.fit(vic_batch_X, vic_batch_y)

    # Create producer and dashboard
    producer = KafkaDataProducer(
        spark_df,
        pre_trained_model_nsw=lr_nsw,
        online_model_nsw=online_model_nsw,
        pre_trained_model_vic=lr_vic,
        online_model_vic=online_model_vic
    )
    producer.start_producing()

    dashboard = RealtimeDashboard(
        pre_trained_models={'NSW': lr_nsw, 'VIC': lr_vic},
        online_models={'NSW': online_model_nsw, 'VIC': online_model_vic}
    )
    dashboard.run_server()

if __name__ == "__main__":
    main(df_processed)